In [1]:
# Thanks to Nicks Niche for the starter code
# in case needed
#!pip install pulp

In [2]:
# Considerations
# - Players are not independent, how to account for stacks, correlations?
# - We are playing against others and have to share the winnings, how to get slight edge?
# -- maybe cross off top picks by position
# -- Limit total salary (eg everyone trying to spend 50k, maybe spend 48)
# - We are passing the mean score, but need to consider variance. Might want more risk to get ahead.
# -- Could vary the projections by x%

In [3]:
import pandas as pd
import numpy as np
from pulp import *

from src.utils import fix_name_mismatches
from src.components.import_data import ImportData
from src.components.optimizer import OptimizeLineup

from input_vars import *

### Read in Draft Kings Players/Salaries and Rankings

In [4]:
gd = ImportData(week, game_mode, rank_src, matchup, add_results)
players = gd.read_dk()
player_rankings = gd.read_rankings(players, adhoc_pt_col=adhoc_pt_col)

In [5]:
# QA check for any large names missing rank data
# May signal a name mismatch across sources
mask = player_rankings['FPPG'].isnull() #& player_rankings['Salary'] > 500
player_rankings[mask].sort_values('Salary', ascending=False).head(5)

,Id,Position,Salary,FPPG,ActualFP


In [6]:
availables = player_rankings.groupby(['Position', 'Id', 'FPPG', 'Salary']).agg('count')
availables = availables.reset_index()

# screening on availables
#mask = availables['Salary'] > 900
#availables = availables[mask]

availables = availables[~availables['Id'].isin(scratches)].reset_index(drop=True)

#check for any force ins not in list
print(f"Names Not Found: {list(set(force_ins) - set(availables['Id']))}")

Names Not Found: []


In [7]:
availables

,Position,Id,FPPG,Salary,ActualFP
0,DST,Browns,9.24,3300,0
1,DST,Chiefs,7.88,2900,0
2,DST,Dolphins,8.94,2800,0
3,DST,Texans,7.71,3000,0
4,FLEX,Alec Ingold,1.46,4000,0
...,...,...,...,...,...
184,WR,Robert Woods,6.38,3600,0
185,WR,Skyy Moore,3.84,3000,0
186,WR,Steven Sims,1.87,3000,0
187,WR,Tyreek Hill,25.09,8700,0


### Run Optimizer

In [8]:
op = OptimizeLineup(game_mode, availables, n_lineups, force_ins, add_results, variability_pctg, salary_cap)

results = op.run_optimizer()
results.head(20).T

Execution time of run_optimizer: 49.88 seconds


,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
1,DST_Dolphins_,DST_Dolphins_,DST_Dolphins_,DST_Dolphins_,DST_Dolphins_,DST_Dolphins_,DST_Dolphins_,DST_Dolphins_,DST_Dolphins_,DST_Dolphins_,DST_Dolphins_,DST_Dolphins_,DST_Dolphins_,DST_Browns_,DST_Dolphins_,DST_Dolphins_,DST_Dolphins_,DST_Dolphins_,DST_Dolphins_,DST_Dolphins_
2,FLEX_Noah_Gray,FLEX_Travis_Kelce,FLEX_Dalton_Schultz,FLEX_Travis_Kelce,FLEX_Noah_Gray,FLEX_David_Njoku,FLEX_Brevin_Jordan,FLEX_Brevin_Jordan,FLEX_Tyreek_Hill,FLEX_Brevin_Jordan,FLEX_Dalton_Schultz,FLEX_Travis_Kelce,FLEX_Tyreek_Hill,FLEX_Brevin_Jordan,FLEX_Brevin_Jordan,FLEX_Nico_Collins,FLEX_Amari_Cooper,FLEX_Dalton_Schultz,FLEX_Amari_Cooper,FLEX_Brevin_Jordan
3,QB_Joe_Flacco,QB_Joe_Flacco,QB_Joe_Flacco,QB_Joe_Flacco,QB_Joe_Flacco,QB_Joe_Flacco,QB_Joe_Flacco,QB_Joe_Flacco,QB_Joe_Flacco,QB_Joe_Flacco,QB_Joe_Flacco,QB_Joe_Flacco,QB_Joe_Flacco,QB_Joe_Flacco,QB_Joe_Flacco,QB_Joe_Flacco,QB_Joe_Flacco,QB_Joe_Flacco,QB_Joe_Flacco,QB_Joe_Flacco
4,RB_De'Von_Achane,RB_De'Von_Achane,RB_De'Von_Achane,RB_De'Von_Achane,RB_De'Von_Achane,RB_De'Von_Achane,RB_De'Von_Achane,RB_De'Von_Achane,RB_De'Von_Achane,RB_De'Von_Achane,RB_De'Von_Achane,RB_De'Von_Achane,RB_De'Von_Achane,RB_De'Von_Achane,RB_De'Von_Achane,RB_De'Von_Achane,RB_De'Von_Achane,RB_De'Von_Achane,RB_De'Von_Achane,RB_De'Von_Achane
5,RB_Jerome_Ford,RB_Jerome_Ford,RB_Jerick_McKinnon,RB_Jerome_Ford,RB_Isiah_Pacheco,RB_Isiah_Pacheco,RB_Jerome_Ford,RB_Jerome_Ford,RB_Jerome_Ford,RB_Jerome_Ford,RB_Jerome_Ford,RB_Jerome_Ford,RB_Jerome_Ford,RB_Jerome_Ford,RB_Jerome_Ford,RB_Jerome_Ford,RB_Jerome_Ford,RB_Jerick_McKinnon,RB_Jerome_Ford,RB_Isiah_Pacheco
6,TE_Travis_Kelce,TE_Noah_Gray,TE_Noah_Gray,TE_Noah_Gray,TE_David_Njoku,TE_Noah_Gray,TE_Travis_Kelce,TE_Travis_Kelce,TE_Noah_Gray,TE_Travis_Kelce,TE_Travis_Kelce,TE_Dalton_Schultz,TE_Noah_Gray,TE_Noah_Gray,TE_Durham_Smythe,TE_Noah_Gray,TE_Noah_Gray,TE_Brevin_Jordan,TE_Noah_Gray,TE_David_Njoku
7,WR_Amari_Cooper,WR_Amari_Cooper,WR_Amari_Cooper,WR_Amari_Cooper,WR_Amari_Cooper,WR_Amari_Cooper,WR_Amari_Cooper,WR_Amari_Cooper,WR_Amari_Cooper,WR_Amari_Cooper,WR_Amari_Cooper,WR_Amari_Cooper,WR_Amari_Cooper,WR_Amari_Cooper,WR_Amari_Cooper,WR_Amari_Cooper,WR_Nico_Collins,WR_Amari_Cooper,WR_Nico_Collins,WR_Amari_Cooper
8,WR_Elijah_Moore,WR_Elijah_Moore,WR_Nico_Collins,WR_Elijah_Moore,WR_Robert_Woods,WR_Robert_Woods,WR_Elijah_Moore,WR_Elijah_Moore,WR_Kadarius_Toney,WR_Elijah_Moore,WR_Elijah_Moore,WR_Elijah_Moore,WR_Kadarius_Toney,WR_Nico_Collins,WR_Nico_Collins,WR_Skyy_Moore,WR_Skyy_Moore,WR_Nico_Collins,WR_Skyy_Moore,WR_Robert_Woods
9,WR_Tyreek_Hill,WR_Tyreek_Hill,WR_Tyreek_Hill,WR_Tyreek_Hill,WR_Tyreek_Hill,WR_Tyreek_Hill,WR_Tyreek_Hill,WR_Tyreek_Hill,WR_Nico_Collins,WR_Tyreek_Hill,WR_Nico_Collins,WR_Nico_Collins,WR_Nico_Collins,WR_Tyreek_Hill,WR_Tyreek_Hill,WR_Tyreek_Hill,WR_Tyreek_Hill,WR_Tyreek_Hill,WR_Tyreek_Hill,WR_Tyreek_Hill
ProjFP,133.11804,133.034067,133.022756,132.966542,132.81905,132.672739,132.613348,132.572903,132.559677,132.545185,132.523519,132.504108,132.481853,132.444983,132.430796,132.394126,132.346585,132.328525,132.214405,132.195686


In [10]:
# Current entry is below
results.loc[14,]

1                DST_Browns_
2         FLEX_Brevin_Jordan
3              QB_Joe_Flacco
4           RB_De'Von_Achane
5             RB_Jerome_Ford
6               TE_Noah_Gray
7            WR_Amari_Cooper
8            WR_Nico_Collins
9             WR_Tyreek_Hill
ProjFP            132.444983
Name: 14, dtype: object